In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

# CPI + CLI DATA COLLECTION


## OECD Composite Leading Indicator

In [ ]:
cli = obb.economy.composite_leading_indicator(start_date='2024-01-01', provider='oecd', country=['united_states'])
cli.to_polars()

## Consumer Price Index

### OECD Consumer Price Index

In [ ]:
oecd_cpi = obb.economy.cpi(start_date='2010-01-01', country='united_states', transform='yoy', provider='oecd',harmonized=False, expenditure='total')
oecd_cpi.results


### FRED Consumer Price Index

In [ ]:
fred_cpi = obb.economy.cpi(start_date='2024-07-01', country='united_states', transform='yoy', provider='fred')
fred_cpi.results

# humblCOMPASS 

In [2]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    # symbols=["AAPL"],
    # interval="1d",
    start_date="2020-01-01",
    end_date="2024-12-01",
    membership="humblPEON"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
compass = toolbox.fundamental.humbl_compass(
    country="united_states",
    chart=True,
    recommendations=True
)



INFO: HumblCompassFetcher || START: fetch_data (sync)
DEBUG: HumblCompassFetcher || Running .transform_query()
INFO: HumblCompassFetcher || CLI start date: 2022-08-02 and CPI start date: 2022-08-02. Dates are adjusted to account for CLI data release lag and z-score calculation window.
DEBUG: HumblCompassFetcher || Running .extract_data()


DEBUG: HumblCompassFetcher || Running .transform_data()
INFO: HumblCompassFetcher || END: fetch_data (sync) - Total time: 0.6160s


In [ ]:
compass.to_polars()

In [ ]:
compass.extra


In [ ]:
compass.show()

In [7]:
compass.warnings

[Warning_(category='HumblDataWarning', message='Start date adjusted to 2022-12-02 based on humblPEON membership (2Y of data).'),
 Warning_(category='HumblDataWarning', message='Z-score defaulted to None. No z-score data will be calculated.')]